In [1]:
########################################################
# run using cellchat environment
########################################################
#library(harmony)
library(RColorBrewer)
library(Seurat)
library(ggplot2)
library(reshape2)
library(viridis)
library(dplyr)
library(Matrix)
library(cowplot)
library(getopt)
library(tidyr)
library(tidyverse)
library(dplyr)
library(ComplexHeatmap)
library(scProportionTest)
library(enrichR)
library(ggpubr)
library(DESeq2)
library(rstatix)
options(future.globals.maxSize = 500 * 1024^3) 

#---------Gene Annotations---------------------------------------
gs <- readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/resources/gene_sets.rds')
annotations <- gs$annotations
s.genes <- gs$s.genes
g2m.genes <- gs$g2m.genes
ig_genes <- gs$ig_genes #length=436
hemogenes <- gs$hemo_genes
protein_coding_genes <- gs$protein_coding_genes
#--- palettes
colors <- readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/resources/colors.rds')
#----------------------------------------------
# function for filtering out mitochondrial, hemoglobin, Ig, ribosomal genes from a list of genes
#----------------------------------------------
filter_genes <- function(genes){
  de_fi1 <- genes[!genes%in% ig_genes & !genes %in% hemogenes]
  de_fim1 <- de_fi1[!str_starts(de_fi1, 'MT-')]
  de_fimr1 <- de_fim1[!str_starts(de_fim1, 'RPL') & !str_starts(de_fim1, 'RPS')]
  genesf <- de_fimr1
  return(genesf)
}
# return only protein-coding genes (excl. mito, hemo, Ig, ribo)
filter_genes_p <- function(genes){
de_fi1 <- genes[!genes%in% ig_genes & !genes %in% hemogenes & genes %in% protein_coding_genes]
  de_fim1 <- de_fi1[!str_starts(de_fi1, 'MT-')]
  de_fimr1 <- de_fim1[!str_starts(de_fim1, 'RPL') & !str_starts(de_fim1, 'RPS')]
  genesf <- de_fimr1
  return(genesf)
}

#-----------------------
setwd("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/MMRF_longitudinal")

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: viridisLite


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


The following object is masked from ‘package:reshape2’:

    smiths


── Attaching core tidyverse packages ────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ lubridate 1.9.4     ✔ stringr   1.5.1
✔ purrr     1.1.0     ✔ tibble    3.3.0
── Conflicts ──────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand()    masks Matrix::expand()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ tidyr::pack(

In [2]:
# longitudinal object
obj <- readRDS('mmrf_longitudinal_noDoublets.rds')


In [4]:
head(obj@meta.data)

,sample_id,orig.ident,nCount_RNA,nFeature_RNA,site,sample_orig_id,batch_num,siteXbatch,RNA_snn_res.0.5,seurat_clusters,⋯,G2M.Score,Phase,old.ident,CC.Difference,nCount_SCT,nFeature_SCT,IRD_subset_score,IRD_subset,IRD_subset_f,celltype_f
,<chr>,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,⋯,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<int>,<dbl>,<fct>,<fct>,<fct>
GGGAGTATCCAGTTCC-1_1284_010761_SM,1284_010761_SM,MMRF,9903,791,EMORY,1284_010761,B2,EMORY_B2,13,13,⋯,0.01252350,G2M,13,-0.05217056,5302,569,1.0000000,LateEry,LateEry,Erythrocyte_IFN
CCACGAGTCACCCTCA-1_1284_010761_SM,1284_010761_SM,MMRF,9978,2909,EMORY,1284_010761,B2,EMORY_B2,7,7,⋯,0.69719738,G2M,7,-0.25171374,5312,2428,0.5431845,EarlyEry,EarlyEry,EB_MKi67_1
TAAGCCAGTCCACTTC-1_1284_010761_SM,1284_010761_SM,MMRF,9899,2650,EMORY,1284_010761,B2,EMORY_B2,7,7,⋯,0.69741525,G2M,7,-0.27604777,5263,2170,0.8678911,LateEry,LateEry,EB_MKi67_1
ATGACCACACATATGC-1_1284_010761_SM,1284_010761_SM,MMRF,9950,2658,EMORY,1284_010761,B2,EMORY_B2,7,7,⋯,0.58559358,G2M,7,-0.32687868,5294,2241,0.6894119,LateEry,LateEry,EB
GGTTAACTCCTTCGAC-1_1284_010761_SM,1284_010761_SM,MMRF,9959,2301,EMORY,1284_010761,B2,EMORY_B2,9,9,⋯,-0.05608178,S,9,0.12818399,4985,1801,1.0000000,Neutrophil,Neutrophil,Neutrophil_RPS/RPL
TTGCGTCTCCGTGGTG-1_1284_010761_SM,1284_010761_SM,MMRF,9892,1576,EMORY,1284_010761,B2,EMORY_B2,14,14,⋯,0.61417626,G2M,14,-0.50606499,5343,1224,1.0000000,LateEry,LateEry,EB_MKi67_1


In [5]:
colnames(obj@meta.data)

[1] "sample_id"                                       
 [2] "orig.ident"                                      
 [3] "nCount_RNA"                                      
 [4] "nFeature_RNA"                                    
 [5] "site"                                            
 [6] "sample_orig_id"                                  
 [7] "batch_num"                                       
 [8] "siteXbatch"                                      
 [9] "RNA_snn_res.0.5"                                 
[10] "seurat_clusters"                                 
[11] "barcode_full"                                    
[12] "percent.mt"                                      
[13] "ambient_rho"                                     
[14] "doublet_score"                                   
[15] "predicted_doublet"                               
[16] "cohort"                                          
[17] "seurat_subclusters_label_transferring_Yizhe_v1"  
[18] "celltype_subclusters_label_transferring_Yizhe_v1"
[19] "seurat_subclusters_integration_Yizhe_v1"         
[20] "celltype_subclusters_integration_Yizhe_v1"       
[21] "compartment"                                     
[22] "UMAP_1"                                          
[23] "UMAP_2"                                          
[24] "public_id"                                       
[25] "collection_event"                                
[26] "S.Score"                                         
[27] "G2M.Score"                                       
[28] "Phase"                                           
[29] "old.ident"                                       
[30] "CC.Difference"                                   
[31] "nCount_SCT"                                      
[32] "nFeature_SCT"                                    
[33] "IRD_subset_score"                                
[34] "IRD_subset"                                      
[35] "IRD_subset_f"                                    
[36] "celltype_f"

In [6]:
unique(obj$celltype_f)

[1] Erythrocyte_IFN      EB_MKi67_1           EB                  
 [4] Neutrophil_RPS/RPL   GMP                  pDCs                
 [7] Pc_MALAT1            Erythrocyte          HSC                 
[10] cDC2                 MAIT                 CD4_Teff            
[13] CD14+Mono_S100       CD14+Mono_IFN        Granulocyte         
[16] CD16+Mono            CD4_Tcm_NFKBIA       Erythrocyte_mito    
[19] CD8_Tn               CD4_Teff_TNF         CD4_Tcm_KLRB1       
[22] CD14+Mono_CTSS       CD4_Tem_IFN          B_preLg             
[25] Pc_MT-CO2            CD14+Mono pro-inflam Treg                
[28] CD8_Tcm_GZMK         CD4_Tn               CD4_Tcm_IFN         
[31] CD4_Th               B_pro                Fibroblasts         
[34] CD8_Teff_TNF         B_mem                CD8_Teff_HLA        
[37] CD8_T_adp            NK_CD56bright        CD8_Tem_NFKB        
[40] NK_CD56dim           CD8_Teff             Pc_JUN              
[43] CD8_Tem_IFN          CD8_Tem              NK_resident         
[46] B_imm                NK_adp               Pc_IGKC             
[49] MegaK                B_naive              CD4_Th_LEF1         
[52] B_preSm              B_trns               CD8_T_Apoptotic     
[55] Erythrocyte_prolif   cDC1                 Pc_HBB_S100A8       
[58] CD8_Teff_b           Macro/Mono           Pc_HBB_HMGB1        
[61] CD4_CTL              CD8_Tcm              CD8_Teff_c          
[64] M2_Macro             EB_MKi67_2           Pc_ATF5             
[67] Pc_IFI27             Pc_EIF1              CD14+Mono_hypo      
[70] Pc_CD74              B_Unswitched_mem     Pc_IGHV3            
[73] Pc_IGHG3             Neutrohpil_ARG1      Pc_PPM1K            
[76] MastC                Pc_NOL4              CD8_T_adp_b         
[79] Pc_CCDC88A           Pc_IGHA1             Pc_BCMA             
[82] Pc_PUS3              Pc_MPO               Pc_IGKV4            
84 Levels: Pc_ATF5 Pc_BCMA Pc_CCDC88A Pc_CD74 Pc_EIF1 ... Fibroblasts

In [14]:
bcells <- grep("^B_", unique(obj$celltype_f), value = TRUE)
bcells

[1] "B_preLg"          "B_pro"            "B_mem"            "B_imm"           
[5] "B_naive"          "B_preSm"          "B_trns"           "B_Unswitched_mem"

In [15]:
b <- subset(obj, subset= celltype_f %in% bcells)

In [20]:
unique(b$collection_event)

[1] "Baseline"            "Remission/Response"  "Relapse/Progression"
[4] "Post-transplant"     "100 Post ASCT"

In [21]:
Idents(b) <- 'collection_event'
b <- RenameIdents(b, 
             '100 Post ASCT'='Post-transplant'
                )
b$collection <- Idents(b)

In [23]:
p <- DotPlot(
  b,
  features = 'SOX4',
  assay = 'RNA',
  cols = c("white", "black"),
  group.by='collection'
)
pdf('sox4expr_mmrf_longitudinal_bcells.pdf', width=5, height=2.5)
print(p)
dev.off()

Warning message:
“Scaling data with a low number of groups may produce misleading results”


agg_record_2070518142 
                    2

In [2]:
tnk <- readRDS('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/MMRF_longitudinal/mmrf_tnk.rds')

In [3]:
ctlevels <- read.table('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/MMRF_longitudinal/mmrf_obj_celltype_levels.txt', sep='\t', header=T, quote='', comment.char='')

tnks <- ctlevels$ct[29:58]
tnks

[1] "CD4_CTL"         "CD4_Tcm_IFN"     "CD4_Tcm_KLRB1"   "CD4_Tcm_NFKBIA" 
 [5] "CD4_Teff"        "CD4_Teff_TNF"    "CD4_Tem_IFN"     "CD4_Th"         
 [9] "CD4_Th_LEF1"     "CD4_Tn"          "Treg"            "CD8_T_adp"      
[13] "CD8_T_adp_b"     "CD8_T_Apoptotic" "CD8_Tcm"         "CD8_Tcm_GZMK"   
[17] "CD8_Teff"        "CD8_Teff_b"      "CD8_Teff_c"      "CD8_Teff_HLA"   
[21] "CD8_Teff_TNF"    "CD8_Tem"         "CD8_Tem_IFN"     "CD8_Tem_NFKB"   
[25] "CD8_Tn"          "MAIT"            "NK_adp"          "NK_CD56bright"  
[29] "NK_CD56dim"      "NK_resident"

In [8]:
# proportion changes over time
tnk_meta <- tnk@meta.data
tnk_meta$sample_id <- paste0(tnk_meta$public_id,"|",tnk_meta$collection_event)
st <- as.data.frame(table(tnk_meta$sample_id))
names(st) <- c("Sample", "nCells_Sample")
sca <- as.data.frame(table(tnk_meta$sample_id, tnk_meta$celltype_f))
colnames(sca) <- c('Sample', 'CT', 'Freq')
scas <- merge(sca, st, by='Sample')
scas$Prop_of_Sample <- scas$Freq/scas$nCells_Sample*100
scas <- scas %>%
  separate(Sample, into = c("public_id", "collection_event"), sep = "\\|")
scas$CT <- factor(scas$CT, levels=ctlevels$ct)
scas$collection_event <- factor(scas$collection_event, levels=c('Baseline', 'Post-transplant', 'Remission/Response', 'Relapse/Progression' ))
head(scas)

,public_id,collection_event,CT,Freq,nCells_Sample,Prop_of_Sample
,<chr>,<fct>,<fct>,<int>,<int>,<dbl>
1,MMRF_1061,Relapse/Progression,CD4_CTL,15,528,2.8409091
2,MMRF_1061,Relapse/Progression,Treg,11,528,2.0833333
3,MMRF_1061,Relapse/Progression,CD8_Teff_TNF,21,528,3.9772727
4,MMRF_1061,Relapse/Progression,CD8_Tem,29,528,5.4924242
5,MMRF_1061,Relapse/Progression,CD4_Th,2,528,0.3787879
6,MMRF_1061,Relapse/Progression,CD4_Tcm_IFN,8,528,1.5151515


In [17]:
unique(scas$collection_event)

[1] Relapse/Progression Remission/Response  Baseline           
[4] <NA>                Post-transplant    
Levels: Baseline Post-transplant Remission/Response Relapse/Progression

In [22]:
df <- scas %>%
  filter(collection_event %in% c("Baseline", "Remission/Response"))
length(unique(df$public_id))

[1] 36

In [25]:
       
paired_ids <- df %>%
  distinct(public_id, collection_event) %>%
  dplyr::count(public_id) %>%
  filter(n == 2) %>%
  pull(public_id)
paired_ids

[1] "MMRF_1284" "MMRF_1290" "MMRF_1311" "MMRF_1332" "MMRF_1352" "MMRF_1389"
 [7] "MMRF_1391" "MMRF_1395" "MMRF_1407" "MMRF_1415" "MMRF_1424" "MMRF_1434"
[13] "MMRF_1454" "MMRF_1466" "MMRF_1474" "MMRF_1496" "MMRF_1518" "MMRF_1534"
[19] "MMRF_1539" "MMRF_1565" "MMRF_1572" "MMRF_1607" "MMRF_1624" "MMRF_1656"
[25] "MMRF_1659" "MMRF_1670" "MMRF_1674" "MMRF_1722" "MMRF_1784" "MMRF_1838"
[31] "MMRF_1839" "MMRF_1854"

In [26]:
df_paired <- scas %>%
  filter(
    public_id %in% paired_ids,
    collection_event %in% c("Baseline", "Remission/Response")
  )
wide <- df_paired %>%
  select(public_id, CT, collection_event, Prop_of_Sample) %>%
  pivot_wider(
    names_from  = collection_event,
    values_from = Prop_of_Sample
  ) %>%
  drop_na(Baseline, `Remission/Response`)


In [28]:

# comparisons
my_comparisons <- list(c("Baseline", "Remission/Response"))

# paired Wilcoxon per CT with FDR
stat.test <- df_paired %>%
  filter(collection_event %in% c("Baseline", "Remission/Response")) %>%
  group_by(CT) %>%
  wilcox_test(
    Prop_of_Sample ~ collection_event,
    paired = TRUE,
    comparisons = my_comparisons
  ) %>%
  adjust_pvalue(method = "fdr") %>%
  add_significance("p.adj")

# keep only significant ones
selected_CTs <- tnks
stat.test.filtered <- stat.test %>%
  filter(CT %in% selected_CTs, p.adj.signif != "ns") %>%
  add_xy_position(x = "collection_event")

plot_df <- df_paired %>%
  filter(
    CT %in% unique(stat.test.filtered$CT),
    collection_event %in% c("Baseline", "Remission/Response")
  )

# paired plot with facets
p <- ggpaired(
  plot_df,
  x = "collection_event",
  y = "Prop_of_Sample",
  id = "public_id",
  line.color = "gray",
  line.size = 0.5,
  dot.size = 3,
  palette = "jco"
) +
  stat_pvalue_manual(stat.test.filtered, label = "p.adj.signif") +
  facet_wrap(~CT, scales = "free_y", ncol = 4) +
  labs(x = "Collection Event", y = "% of total T/NK") +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)
  )

pdf("mmrf_TNK_proportions_overTime_paired.pdf", width = 8, height = 12)
print(p)
dev.off()

agg_record_2051180824 
                    2